# Obteniendo los datos estadisticos de todos los archivos

Se calculan todas las variables estadisticas y se almacenan en un dataframe

### Importando librerias

In [1]:
import pandas as pd
from scipy import stats
from sklearn.preprocessing import MinMaxScaler
import math
import numpy as np
import matplotlib.pyplot as plt

### Importar lista con nombres de archivos

*Cambiar el nombre de la lista según el sentimiento a analizar

In [2]:
files = pd.read_csv("Happiness.csv") ### MODIFICAR EL NOMBRE

### Se crea un diccionario para almacenar las variables de cada archivo

In [3]:
data_instance = {'M1': [],
                 'M2': [],
                 'M3': [],
                 'M4': [],
                 'C2': [],
                 'C3': [],
                 'C4': [],
                 'Skewness': [],
                 'Kurtosis': [],
                 'Mean': [],
                 'Absolute_mean': [],
                 'Peak_value': [],
                 'SRV': [],
                 'RMS': [],
                 'Crest_factor': [],
                 'Shape_factor': []}

### Ciclo para leer todos los archivos de cada sentimiento

In [4]:
for a in range(int(files.count())):
    name_doc = files._get_value (a, 'Name')
    data = pd.read_csv (name_doc)



    ########################################################################################################################################
    ################# Fourier #################
    raw_data = np.ravel(data.to_numpy())
    n = len(raw_data)
    yf = np.fft.fft(raw_data)       #Get fourier transform
    yfts = np.fft.fftshift(yf)
    yftsa = 1.0/n * np.abs(yfts)        #Remove complex numbers

    # Convert numpy array to DataFrame
    data = pd.DataFrame (yftsa)
    ##########################################################################################################################################



    ################# Get statistical features #################
    m1 = float (stats.moment (data, moment = 1)) #First moment 
    m2 = float (stats.moment (data, moment = 2)) #Second moment
    m3 = float (stats.moment (data, moment = 3))#Third moment
    m4 = float (stats.moment (data, moment = 4)) #Fourth moment
    sc = float (m2 - math.sqrt (pow (m1,2))) #Second cumulant
    tc = float (m3 - (3*m1*m2) - (2*pow (m1,3))) #Third cumulant
    fc = float (m4 + (3*m3*m1) - (3*pow (m2,2)) + (12*m2*pow (m1,2)) - (6*pow (m1,4))) #Fourth cumulant
    skew = float (stats.skew (data)) #Skewness
    kurt = float (stats.kurtosis (data)) #Kurtosis
    mean = float(data.mean()) #Mean
    am = abs(float(data.mean())) #Absolute mean
    pv = 0.5*(float(data.max()) - float(data.min())) #Peak value
    da = abs(data)
    dsa = da**2
    srv = float(pow((dsa.sum()/int(dsa.count())),2)) #Square root value
    d2 = data**2
    rms = math.sqrt (float(d2.sum())/int(d2.count())) #Root mean square
    cf = pv/rms #Crest factor
    sf = rms/am #Shape factor

    ################# Append features to a dictionary #################
    data_instance ['M1'].append(m1)
    data_instance ['M2'].append(m2)
    data_instance ['M3'].append(m3)
    data_instance ['M4'].append(m4)
    data_instance ['C2'].append(sc)
    data_instance ['C3'].append(tc)
    data_instance ['C4'].append(fc)
    data_instance ['Skewness'].append(skew)
    data_instance ['Kurtosis'].append(kurt)
    data_instance ['Mean'].append(mean)
    data_instance ['Absolute_mean'].append(am)
    data_instance ['Peak_value'].append(pv)
    data_instance ['SRV'].append(srv)
    data_instance ['RMS' ].append(rms)
    data_instance ['Crest_factor'].append(cf)
    data_instance ['Shape_factor'].append(sf)

    del data

### Se crea un dataframe con todos los datos estadisticos y se guarda en un .csv

In [5]:
################# Create a dataframe from dictionary #################
data_stats = pd.DataFrame(data=data_instance)
data_stats.head()

,M1,M2,M3,M4,C2,C3,C4,Skewness,Kurtosis,Mean,Absolute_mean,Peak_value,SRV,RMS,Crest_factor,Shape_factor
0,0.0,2.124380e-07,1.552423e-09,2.795212e-11,2.124380e-07,1.552423e-09,2.781673e-11,15.854841,616.370561,0.000079,0.000079,0.021391,4.779280e-14,0.000468,45.750727,5.948776
1,0.0,2.189161e-07,1.485608e-09,2.401105e-11,2.189161e-07,1.485608e-09,2.386727e-11,14.504001,498.020872,0.000087,0.000087,0.019767,5.125665e-14,0.000476,41.544311,5.500377
2,0.0,3.218549e-07,2.463789e-09,3.895685e-11,3.218549e-07,2.463789e-09,3.864608e-11,13.493148,373.065619,0.000100,0.000100,0.019804,1.100750e-13,0.000576,34.382311,5.782968
3,0.0,2.991077e-07,2.067720e-09,2.963744e-11,2.991077e-07,2.067720e-09,2.936905e-11,12.640098,328.272691,0.000103,0.000103,0.019763,9.586097e-14,0.000556,35.517156,5.428507
4,0.0,2.515135e-07,1.771296e-09,2.448957e-11,2.515135e-07,1.771296e-09,2.429980e-11,14.042658,384.131679,0.000085,0.000085,0.019736,6.696935e-14,0.000509,38.796405,5.965893


In [6]:
#Normalizacion
scaler = MinMaxScaler()
data_stats = pd.DataFrame (scaler.fit_transform(data_stats),columns=['M1','M2','M3','M4','C2','C3','C4','Skewness','Kurtosis','Mean','Absolute_mean','Peak_value','SRV','RMS','Crest_factor','Shape_factor'])

data_stats.to_csv (r"..\stats\Normalizated_data\Stats_happiness_fourier.csv")
data_stats.head()

,M1,M2,M3,M4,C2,C3,C4,Skewness,Kurtosis,Mean,Absolute_mean,Peak_value,SRV,RMS,Crest_factor,Shape_factor
0,0.0,0.244491,0.128644,0.112029,0.244491,0.128644,0.112503,0.420360,0.203469,0.433148,0.433148,0.972245,0.059387,0.480580,0.611249,0.569578
1,0.0,0.251966,0.123107,0.096233,0.251966,0.123107,0.096530,0.349742,0.158742,0.479290,0.479290,0.895925,0.063691,0.489517,0.537265,0.420870
2,0.0,0.370742,0.204172,0.156134,0.370742,0.204172,0.156302,0.296898,0.111519,0.555715,0.555715,0.897655,0.136778,0.598024,0.411296,0.514589
3,0.0,0.344495,0.171348,0.118783,0.344495,0.171348,0.118782,0.252304,0.094591,0.572630,0.572630,0.895710,0.119116,0.576828,0.431256,0.397035
4,0.0,0.289578,0.146783,0.098151,0.289578,0.146783,0.098279,0.325625,0.115702,0.472076,0.472076,0.894453,0.083215,0.525143,0.488933,0.575255
